### Loading the required libraries

In [4]:
import evaluate
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer

### Creating a function to evaluate model accuracy

In [5]:
metric = evaluate.load("accuracy")

In [6]:
def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

### Loading the dataset for sentiment analysis

In [7]:
dataset = load_dataset("mteb/tweet_sentiment_extraction")

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00001.parquet:   0%|          | 0.00/195k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

### Tokenizing the data

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

### Splitting the dataset into training and testing datasets.

In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Loading GPT2  model

In [11]:
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Finetuning the model

In [ ]:

training_args = TrainingArguments(
   output_dir="test_trainer",
   eval_strategy="steps",
   eval_steps=100,
   per_device_train_batch_size=1,  
   per_device_eval_batch_size=1,  
   gradient_accumulation_steps=1,
   num_train_epochs=1,
   )


In [15]:

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,
   
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,1.130218,0.409000
200,No log,2.308961,0.457000
300,No log,2.391252,0.583000
400,No log,2.105901,0.599000
500,1.560400,1.792854,0.677000
600,1.560400,2.225043,0.657000
700,1.560400,1.518890,0.714000
800,1.560400,1.744574,0.682000
900,1.560400,1.702598,0.713000
1000,1.582900,1.829990,0.703000


TrainOutput(global_step=1000, training_loss=1.5716746826171875, metrics={'train_runtime': 1128.0838, 'train_samples_per_second': 0.886, 'train_steps_per_second': 0.886, 'total_flos': 522598219776000.0, 'train_loss': 1.5716746826171875, 'epoch': 1.0})

### Evaluate the model

In [16]:
trainer.evaluate()

{'eval_loss': 1.8299897909164429,
 'eval_accuracy': 0.703,
 'eval_runtime': 70.8858,
 'eval_samples_per_second': 14.107,
 'eval_steps_per_second': 14.107,
 'epoch': 1.0}